In [1]:
import numpy as np
import pandas as pd

In [2]:
%%bash
cd ../third-party/COVID-19
git reset --hard HEAD^
git clean -fd
git fetch
git rebase origin/master

HEAD is now at 1f099251 automated update
First, rewinding head to replay your work on top of it...
Fast-forwarded master to origin/master.


In [3]:
folder_path      = '../third-party/COVID-19/csse_covid_19_data/csse_covid_19_time_series'
deaths_global    = pd.read_csv('%s/time_series_covid19_deaths_global.csv' % folder_path)
confirmed_global = pd.read_csv('%s/time_series_covid19_confirmed_global.csv' % folder_path)
recovered_global = pd.read_csv('%s/time_series_covid19_recovered_global.csv' % folder_path)

In [4]:
deaths_global    = deaths_global.groupby("Country/Region").sum()
confirmed_global = confirmed_global.groupby("Country/Region").sum()
recovered_global = recovered_global.groupby("Country/Region").sum()

In [5]:
deaths_global    = deaths_global.drop(columns=['Lat', 'Long'])
confirmed_global = confirmed_global.drop(columns=['Lat', 'Long'])
recovered_global = recovered_global.drop(columns=['Lat', 'Long'])

In [6]:
shared_index = sorted(set(deaths_global.index) & set(confirmed_global.index) & set(recovered_global.index))

In [7]:
deaths_global    = confirmed_global.loc[shared_index]
confirmed_global = confirmed_global.loc[shared_index]
recovered_global = recovered_global.loc[shared_index]

In [8]:
deaths_global.index.name    = 'Country'
confirmed_global.index.name = 'Country'
recovered_global.index.name = 'Country'

In [9]:
deaths_global.columns    = sorted(pd.to_datetime(deaths_global.columns))
confirmed_global.columns = sorted(pd.to_datetime(confirmed_global.columns))
recovered_global.columns = sorted(pd.to_datetime(recovered_global.columns))

In [10]:
matrix = \
    np.transpose(
        np.stack([
            deaths_global.values,    # 0: Number of Deaths
            confirmed_global.values, # 1: Number of Confirmed Cases
            recovered_global.values  # 2: Number of Recovered People
        ]), (1, 2, 0)
    )

In [11]:
np.save("../prepared-data/COVID-19 (%03dx%03dx%02d)" % matrix.shape, matrix)